In [1]:
import sys
import os
import tensorflow as tf
import numpy as np
from utils import *
import matplotlib.pyplot as plt
from datetime import datetime

from utils import *
from equations import *

2023-10-20 17:27:06.098310: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 17:27:06.223706: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-20 17:27:06.227059: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /totten_1/chenggong/trunk-jpl/externalpackages/netcdf/install/lib:/totten_1/chenggo

In [2]:
epochADAM = 1000
epochLBFGS=0
N_u=50
N_f=100
seed=1234
log_frequency=10000
history_frequency=10
NLayers=6
NNeurons=10
noiseLevel=[]
# vel, (h,H), C, FSSA, (FH, SMB), BC
weights = [5,3,5,8,1,14]

In [3]:
hp = {}
# Data size on the solution u
hp["N_u"] = N_u
# Collocation points size, where we’ll check for f = 0
hp["N_f"] = N_f
# DeepNN topology (2-sized input [x,t], NLayers hidden layer of NNeurons-width, 1-sized output [u]
hp["layers"] = [2]+[NNeurons]*NLayers+[1]
# DeepNN topology (2-sized input [x,t], NLayers hidden layer of NNeurons-width, 2-sized output [h, H]
hp["h_layers"] = [2]+[NNeurons]*NLayers+[2]
# DeepNN topology (1-sized input [x], NLayers hidden layer of NNeurons-width, 1-sized output [C]
hp["C_layers"] = [1]+[NNeurons]*NLayers+[1]
# DeepNN topology (2-sized input [x,t], NLayers hidden layer of NNeurons-width, 1-sized output [SMB]
hp["smb_layers"] = [2]+[NNeurons]*NLayers+[1]

# Setting up the TF SGD-based optimizer (set tf_epochs=0 to cancel it)
hp["tf_epochs"] = epochADAM
hp["tf_lr"] = 0.001
hp["tf_b1"] = 0.99
hp["tf_eps"] = 1e-1
# Setting up the quasi-newton LBGFS optimizer (set nt_epochs=0 to cancel it)
hp["nt_epochs"] = epochLBFGS
hp["log_frequency"] = log_frequency
# Record the history
hp["save_history"] = True
hp["history_frequency"] = history_frequency
# path for loading data and saving models
repoPath = "./"
appDataPath = os.path.join(repoPath, "matlab_SSA", "DATA")
path = os.path.join(appDataPath, "Helheim_Big_Rignot_SMW_Schoof_PINN_flowline_transient.mat")

loss_weights = [10**(-w) for w in weights]

now = datetime.now()

modelPath = "./Models/SSA1D_transient_"+str(NLayers)+"x"+str(NNeurons)+"_weights"+ "".join([str(w)+"_" for w in weights]) + now.strftime("%Y%m%d_%H%M%S")

modelPath += ("_seed_" + str(seed) if seed else "")
# + "ADAM"+str(hp["tf_epochs"]) +"_BFGS"+str(hp["nt_epochs"])
reloadModel = False # reload from previous training

In [4]:
X_star, u_star, X_train, u_train, X_1d, C_1d, X_f, X_bc, u_bc, X_cf, n_cf, xub, xlb, uub, ulb, mu = prep_Helheim_transient(path=path, N_u=N_u, N_f=N_f)

In [5]:
logger = Logger(hp)
pinn = SSA1D_transient(hp, logger, X_f,
        X_bc, u_bc,
        X_cf, n_cf,
        xub, xlb, uub, ulb,
        modelPath, reloadModel,
        mu=mu,
        loss_weights=loss_weights)

Hyperparameters:
{
  "N_u": 50,
  "N_f": 100,
  "layers": [
    2,
    10,
    10,
    10,
    10,
    10,
    10,
    1
  ],
  "h_layers": [
    2,
    10,
    10,
    10,
    10,
    10,
    10,
    2
  ],
  "C_layers": [
    1,
    10,
    10,
    10,
    10,
    10,
    10,
    1
  ],
  "smb_layers": [
    2,
    10,
    10,
    10,
    10,
    10,
    10,
    1
  ],
  "tf_epochs": 1000,
  "tf_lr": 0.001,
  "tf_b1": 0.99,
  "tf_eps": 0.1,
  "nt_epochs": 0,
  "log_frequency": 10000,
  "save_history": true,
  "history_frequency": 10
}

TensorFlow version: 2.11.0
Eager execution: True
GPU-accerelated: []


2023-10-20 17:27:08.119423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /totten_1/chenggong/trunk-jpl/externalpackages/netcdf/install/lib:/totten_1/chenggong/trunk-jpl/externalpackages/petsc/install/lib:/totten_1/chenggong/trunk-jpl/externalpackages/hdf5/install/lib:/totten_1/chenggong/trunk-jpl/externalpackages/proj/install/lib:/totten_1/chenggong/trunk-jpl/externalpackages/gdal/install/lib:/totten_1/chenggong/trunk-jpl/externalpackages/triangle/install/lib:/totten_1/chenggong/Elmer/install/lib:
2023-10-20 17:27:08.119441: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-20 17:27:08.119456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (totten): /proc/driver/n

In [6]:
# error function for logger
X_u = pinn.tensor(X_1d)
u = pinn.tensor(C_1d)
def error():
    return pinn.test_error(X_u, u)
logger.set_error_fn(error)

In [7]:
pinn.fit(X_train, u_train)


Training started
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 minmax_scale_layer (MinmaxS  (None, 2)                6         
 caleLayer)                                                      
                                                                 
 dense (Dense)               (None, 10)                30        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                      

ValueError: in user code:

    File "/totten_1/chenggong/PINNs/utils/neuralnetwork.py", line 219, in Adam_optimization_step  *
        loss_value, grads = self.grad(X_u, u)
    File "/totten_1/chenggong/PINNs/utils/neuralnetwork.py", line 119, in grad  *
        loss_value = self.loss(u, X)
    File "/totten_1/chenggong/PINNs/equations/SSA1D_transient.py", line 193, in loss  *
        fc1_pred = self.cf_model(self.X_cf, self.n_cf)
    File "/totten_1/chenggong/PINNs/equations/SSA1D_transient.py", line 150, in cf_model  *
        u, s, H, C, smb, u_x, H_x, H_t = self.nn_model(X)
    File "/totten_1/chenggong/PINNs/equations/SSA1D_transient.py", line 69, in nn_model  *
        x = X[:, 0:1]

    ValueError: Index out of range using input dim 1; input has only 1 dims for '{{node strided_slice}} = StridedSlice[Index=DT_INT32, T=DT_DOUBLE, begin_mask=1, ellipsis_mask=0, end_mask=1, new_axis_mask=0, shrink_axis_mask=0](X, strided_slice/stack, strided_slice/stack_1, strided_slice/stack_2)' wi

In [ ]:
plot_log_history(pinn)

In [ ]:
# pinn.save()
# # plot
# plot_1D_solutions_all(pinn, X_f, X_star, u_star, xlb, xub, savePath=modelPath)
# # history
# plot_log_history(pinn, modelPath)